In [4]:
%cd /content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code/


/content/drive/MyDrive/WorkSpace/AI_Research/EmotionTorch/code


In [ ]:
!pip install pytorchcv -q
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from utils.datasets.fer2013_ds import FERDataset
# from models.vgg16_cbam import  VGG19_CBAM
from models.resnet_cbam import ResidualNet , cbam_resnet50
from models.vggnet import vgg16_bn, vgg19, vgg19_bn, vgg16
from models.resnet import resnet50
from models.vggnet_cbam import vgg16_cbam, vgg19_cbam
from models.vggnet_cbam_pre import *
from torch.utils.data import DataLoader

from utils.visualize.show_img import show_image_dataset
from trainer.fer2013_trainer import FER2013_Trainer
import json

import matplotlib.pyplot as plt 
import cv2
import numpy as np

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

try:
    from torch.hub import load_state_dict_from_url
except ImportError:
    from torch.utils.model_zoo import load_url as load_state_dict_from_url

from models.cbam import *


model_urls = {
    "vgg11": "https://download.pytorch.org/models/vgg11-bbd30ac9.pth",
    "vgg13": "https://download.pytorch.org/models/vgg13-c768596a.pth",
    "vgg16": "https://download.pytorch.org/models/vgg16-397923af.pth",
    "vgg19": "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth",
    "vgg11_bn": "https://download.pytorch.org/models/vgg11_bn-6002323d.pth",
    "vgg13_bn": "https://download.pytorch.org/models/vgg13_bn-abd245e5.pth",
    "vgg16_bn": "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth",
    "vgg19_bn": "https://download.pytorch.org/models/vgg19_bn-c79401a0.pth",
}

vgg16_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512,
                'M', 512, 512, 512, 'M']

vgg19_config = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512,
                512, 512, 'M', 512, 512, 512, 512, 'M']

flat = [64*224*224, 128*112*112, 256*56*56, 512*28*28, 512*14*14]

class Cbam(nn.Module):
    def __init__(self, in_channels):
        super(Cbam, self).__init__()

        self.Cbam = CBAM(in_channels)
        self.relu = nn.ReLU(inplace = True)

    def forward(self, x):
        x = self.Cbam(x)
        x = self.relu(x)
        return x

class ResCbam(nn.Module):
    def __init__(self, in_channels):
        super(ResCbam, self).__init__()

        self.Cbam = CBAM(in_channels)
        self.relu = nn.ReLU(inplace = True)

    def forward(self, x):
        res = x
        x = self.Cbam(x)
        # x = x*(1+res)
        x += res
        x = self.relu(x)
        return x   

class Dense(nn.Module):
    def __init__(self, in_features):
      super(Dense, self).__init__()

      self.in_features = in_features

      self.fc = nn.Linear(self.in_features, 256)
      self.relu = nn.ReLu(inplace = True)

    def forward(self, x):
      x = self.fc(x)
      x = self.relu(x)

      return x


class BackBone(nn.Module):
    def __init__(self, layers):
        super().__init__()
        
        self.layers = layers

        self.features = nn.Sequential(*self.layers)
        
        self.avgpool = nn.AdaptiveAvgPool2d(7)

        self.fc_layers = [
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace = True),
            nn.Dropout(0.5),
            nn.Linear(4096, 1000)
        ]
        self.classifier = nn.Sequential(*self.fc_layers)
        # self._initialize_weights()
        # print(self.features)
    def forward(self, x):
       
        x = self.features(x)
        # x = layers(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        
        return x
    
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode="fan_out", nonlinearity="relu")
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)



def make_vgg_layers(vgg_config, batch_norm):

    layers = []
    in_channels = 3

    for c in vgg_config:
        assert c == "M" or isinstance(c, int)
        if c == "M":
            layers += [nn.MaxPool2d(kernel_size = 2)]
        
        else:
            conv2d =  (nn.Conv2d(in_channels, c, kernel_size = 3, padding = 1))
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(c), nn.ReLU(inplace = True)]
            else:
                layers += [conv2d, nn.ReLU(inplace = True)]

            in_channels = c

    return layers

def get_output(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

class VGGBN16CbamMultiFc(BackBone):
    def __init__(self):
        super(VGGBN16CbamMultiFc, self).__init__(make_vgg_layers(vgg16_config, batch_norm =  True))

        state_dict = load_state_dict_from_url(model_urls["vgg16_bn"], progress = True)
        self.load_state_dict(state_dict)

        k = 0
        for i in range(len(vgg16_config)):
            if vgg16_config[i] == "M":
                # k +=1 
                self.layers.insert(k, ResCbam(vgg16_config[i-1]))
                k += 2
            else:
                k += 3
        

        self.features = nn.Sequential(*self.layers)
        self.classifier = nn.Sequential(*self.fc_layers)

    def forward(self, x):
        
        x = self.features(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)

        return x 
        
class TestModel(nn.Module):
    def __init__(self, model):
      super(TestModel, self).__init__()

      self.feature1 = torch.nn.Sequential(*list(model.features.children())[:7])
      self.feature1 = torch.nn.Sequential(*list(model.features.children())[:7])
      self.feature1 = torch.nn.Sequential(*list(model.features.children())[:7])
      self.feature1 = torch.nn.Sequential(*list(model.features.children())[:7])
      self.feature1 = torch.nn.Sequential(*list(model.features.children())[:7])


    def forward(self, x):
      activation = {}
      k = 0
      c = 0
      for i in range(len(vgg16_config)):
        if vgg16_config[i] == "M":
          
          self.model.features[k].register_forward_hook(get_output("features[{}]".format(k)))
          output = self.model(x)
          out = activation["features[{}]".format(k)].shape[1:]
          m = torch.flatten(out, 1)
          self.layers_con.append(Dense(m))
        else:
          k += 3
      
      concat = self.layers_con[0]
      for i in range(len(self.layers_con)-1):

        concats = torch.cat([concat, layers[i+1]])
        concat = concats
      
      return concat

ModuleNotFoundError: ignored